3D_Relaxation
======

This notebook reproduce the a cosine perturbation of the surface relaxation with true free surface. Loading of Earth's surface can be described with an initial periodic surface displacement of a viscous fluid within an infinite half space, the solution of which is outlined in Turcotte and Schubert (1982), 6.10 Postglacial Rebound.  The surface decreases exponentially with time and is dependent on the magnitude, $w_m$, and wavelength $\lambda$ of the perturbation, and the viscosity, $\eta$ and density, $\rho$ of the fluid,

$$ w = w_m exp\Big(\frac{-\lambda \rho g t}{4\pi\eta}\Big) $$

where $w$ is displacement, $w_m$ the initial load magnitude, $g$ gravity, $t$ time. This solution can be charaterised by the relaxation time, $t_{relax} = 4\pi\eta / \rho g \lambda $, the time taken for the initial load to decrease by $e^{-1}$. The solution for an elastic material with the equivalent load produces the same magnitude of displacement instantaneously.


**Keywords:** Free surface

<img src="./images/3D_FreeSurface.gif" width = "400" height = "200"  align=center /> 
<img src="./images/3D_FreeSurface_Topography.png" width = "400" height = "300"  align=center />

In [ ]:
from underworld import UWGeodynamics as GEO
from underworld import visualisation as vis
import underworld as uw
import numpy as np

In [ ]:
u = GEO.UnitRegistry
ndim = GEO.non_dimensionalise
dim = GEO.dimensionalise

# scaling 3: vel
half_rate = 1.0 * u.centimeter / u.year
model_length = 100. * u.kilometer
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

In [ ]:
zres = 8
xres = zres*4
yres = zres*4
figsize = (800,400)

xmin, xmax = -200 * u.kilometer, 200 * u.kilometer
ymin, ymax = -200 * u.kilometer, 200 * u.kilometer
zmin, zmax = -100 * u.kilometer, 0 * u.kilometer

eta = ndim(1e21  * u.pascal * u.second)
density = ndim(3300 * u.kilogram / u.metre**3)
gravity = ndim(9.81 * u.meter / u.second**2)
w_m    =   ndim(5.0 * u.kilometer)
Lambda = ndim(100.0 * u.kilometer) 

densityM = density
viscM = eta
ND_gravity = gravity

def perturbation(x):
    return w_m * np.cos(2.*np.pi*(x)/Lambda)

# analytic solution
xMax = ndim(xmax - xmin)
x = np.linspace(0, xMax, 200+1)
w_0 = perturbation(x)
t_relax = 4 * np.pi * eta / (Lambda * density * gravity)
tMax = t_relax * 5 
t = np.linspace(0, tMax, 100 * 10 + 1)
w_t = w_m * np.exp(-1.*t/t_relax)

max_time =  dim(tMax,u.kiloyear)
dt_set = dim(t_relax*1e-2,u.kiloyear)
save_every = 5
checkpoint_interval = dt_set*save_every

Model = GEO.Model(elementRes=(xres,yres,zres),
                  minCoord=(xmin,ymin,zmin),  
                  maxCoord=(xmax,ymax,zmax),
                  gravity=(0.0, 0.0,-9.81 * u.meter / u.second**2))

fdir = "1_23_05_FreeSurface_3D_Relaxation_zres"+str(zres)+"/"
Model.outputDir = fdir

In [ ]:
MShape = GEO.shapes.Layer3D(top=Model.top,bottom=Model.bottom)
ma = Model.add_material(name="material",shape=MShape)

In [ ]:
def perturbation3D(x,y):
    return w_m * np.cos(2.*np.pi*(x)/Lambda)

minCoord = tuple([GEO.nd(val) for val in Model.minCoord])
maxCoord = tuple([GEO.nd(val) for val in Model.maxCoord])

init_mesh = uw.mesh.FeMesh_Cartesian(elementType=Model.elementType,
                                    elementRes=Model.elementRes,
                                    minCoord=minCoord,
                                    maxCoord=maxCoord,
                                    periodic=Model.periodic)

TField = init_mesh.add_variable(nodeDofCount=1)
TField.data[:, 0] = init_mesh.data[:, -1].copy()

top = Model.top_wall
bottom = Model.bottom_wall
conditions = uw.conditions.DirichletCondition(variable=TField,indexSetsPerDof=(top + bottom,))
system = uw.systems.SteadyStateHeat(
    temperatureField=TField,
    fn_diffusivity=1.0,
    conditions=conditions)
solver = uw.systems.Solver(system)

x = init_mesh.data[top,0]
y = init_mesh.data[top,1]
TField.data[top, 0] = perturbation3D(x,y)

solver.solve()
with Model.mesh.deform_mesh():
     Model.mesh.data[:, -1] = TField.data[:, 0].copy()

Model.population_control.repopulate()

In [ ]:
# Fig = vis.Figure(resolution=figsize,rulers=True,margin = 80,axis=True)
# Fig.Mesh(Model.mesh)
# lv = Fig.window()
# lv.rotate('x',-45)
# lv.redisplay()

In [ ]:
ma.density = 3300. * u.kilogram / u.metre**3
ma.viscosity  =  1e21 * u.pascal * u.second
Model.set_velocityBCs(left=[0.0, None, None],right=[0.0,None, None],
                       front=[None, 0.0, None], back=[None, 0.0, None],
                       bottom=[0.0, 0.0, 0.0],)
Model.freeSurface = True

In [ ]:
# Fig = vis.Figure(resolution=figsize,rulers=True,margin = 80,axis=True)
# Fig.Points(Model.swarm, Model.materialField,fn_size=2.0,discrete=True,colourBar=False)
# lv = Fig.window()
# lv.rotate('x',-45)
# lv.redisplay()

In [ ]:
Model.solver.set_inner_method("mg")

In [ ]:
Model.run_for(max_time, checkpoint_interval=checkpoint_interval,dt= dt_set)